In [14]:
from collections import Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

max_sentence_len = 25

In [ ]:
corpus_movie_conv = "./data/Cornell Movie Dataset/movie_conversations.txt"
corpus_movie_lines = "./data/Cornell Movie Dataset/movie_lines.txt"

In [ ]:
with open(corpus_movie_conv, 'r') as c:
    conv = c.readlines()

In [ ]:
with open(corpus_movie_lines, 'r', encoding="latin1") as l:
    lines = l.readlines()

In [ ]:
lines[0].split(" +++$+++ ")

In [ ]:
lines_dict = {}
for line in lines:
    objects = line.split( ' +++$+++ ')
    lines_dict[objects[0]] = objects[-1]

In [ ]:
def remove_punc(string):
    punctuations = '''!()-[]{};:\,{}./'@#$%^&*_~"'''
    no_punctuation = ""
    for char in string:
        if char not in punctuations:
            no_punctuation += char
    return no_punctuation.lower()

In [ ]:
pairs = []

for con in conv:
    ids = eval(con.split(' +++$+++ ')[-1])
    for i in range(len(ids)):
        qa_pairs = []

        if i == len(ids) - 1:
            break

        first = remove_punc(lines_dict[ids[i]].strip())
        second = remove_punc(lines_dict[ids[i+1]].strip())

        qa_pairs.append(first.split()[:max_sentence_len])
        qa_pairs.append(second.split()[:max_sentence_len])

        pairs.append(qa_pairs)

In [ ]:
word_freq = Counter()

for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])


In [ ]:
min_word_freq = 5
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map['<unk>'] = len(word_map) + 1
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

In [ ]:
print(f"Total words are {len(word_map)}")

In [ ]:
with open('./data/WORDMAP_corpus.json', 'w') as j:
    json.dump(word_map, j)

In [ ]:
def encode_question(words, word_map):
    return [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_sentence_len - len(words))

In [ ]:
def encode_reply(words, word_map):
    return [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<end>']] + [word_map['<pad>']] * (max_sentence_len - len(words))

In [ ]:
pairs_encoded = []
for pair in pairs:
    question = encode_question(pair[0], word_map)
    answer = encode_reply(pair[1], word_map)
    pairs_encoded.append([question, answer])

In [ ]:
with open('./data/pairs_encoded.json', 'w') as j:
    json.dump(pairs_encoded, j)

In [10]:
class Dataset(Dataset):

    def __init__(self):
        self.pairs = json.load(open('./data/pairs_encoded.json'))
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):
        
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
            
        return question, reply

    
    def __len__(self):
        return self.dataset_size


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size=100,
                                           shuffle=True,
                                           pin_memory=True)

In [12]:
def create_masks(question, reply_input, reply_target):

    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)

    question_mask = (question != 0).to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)

    reply_input_mask = reply_input != 0
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data)
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_target_mask = reply_target != 0

    return question_mask, reply_input_mask, reply_target_mask


In [ ]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model, max_len = 50):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embeddings(vocab_size, d_model)
        self.pe = self.create_positional_encoding(max_sentence_len, d_model)

    def create_positional_encoding(self, max_sentence_len, d_model):
        pe = torch.zeros(max_sentence_len, d_model).to(device)
        for pos in range(max_sentence_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10_000 ** ((2 * i) / d_model)))
                pe[pos, i + 1] = math.cos(pos / (10_000 ** ((2 * (i + 1)) / d_model)))
        pe = pe.unsqueeze(0)
        return pe

    def forward(self, encoded_words):
        embeddings = self.embed(encoded_words) * math.sqrt(self.d_model) # (batch_size, max_words, d_model)
        max_words = embeddings.size(1)
        embeddings += self.pe[:, :max_words] # pe will automatically be expanded to same batch_size as embeddings
        embeddings = self.dropout(embeddings)
        return embeddings

All the code past here went past my head

In [ ]:
class MultiHeadAttention(nn.Module):
    
    def __init(self, heads, d_model):
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model) # input and output dimensions
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):
        """
        query, key and value of shape (batch_size, max_words, 512)
        mask: (batch_size, 1, 1, max_words)
        """
        query = self.query(query) # (batch_size, max_words, 512)
        key = self.key(key) # (batch_size, max_words, 512)
        value = self.value(value) # (batch_size, max_words, 512)

        # (batch_size, max_words, 512) => (batch_size, max_words, 8, 64) => (batch_size, h, max_words, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)

        # (batch_size, 8, max_words, d_k) dot (batch_size, 8, d_k, max_words) => (batch_size, 8, max_words, max_words)
        scores = torch.matmul(query, key.permute(0, 1, 3, 2)) / math.sqrt(query.size(-1))
        scores = scores.masked_fill(mask == 0, -1e9)
        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)

        # (batch_size, 8, max_words, max_words) dot (batch_size, 8, max_words, d_k) => (batch_size, 8, max_words, d_k)
        context = torch.matmul(weights, value)
        # (batch_size, 8, max_words, d_k) => (batch_size, max_words, 8, d_k) => (batch_size, max_words, 8 * d_k)
        context = context.permute(0, 2, 1, 3).viwer(context.shape[0], -1, self.heads * self.d_k)
        interacted = self.concat(context)

        return interacted

In [15]:
class FeedForward(nn.Module):
    def __init__(self, d_model, middle_dim=2048):
        super(FeedForward, self).__init__()

        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()

        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.layernorm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

In [ ]:
class DecoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.layernorm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded


In [ ]:
class Transformer(nn.Module):

    def __init__(self, d_model, heads, num_layers, word_map):
        super(Transformer, self).__init__()

        self.d_model = d_model
        self.vocab = len(word_map)
        self.embed = Embeddings(self.vocab, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab)

    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings

    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings

    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decoder(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded))
        return out

In [ ]:
class AdamWarmup:

    def __init__(self, model_size, warmup_steps, optimizer):
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0

    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))

    def step(self):
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        self.lr = lr

        # Update weights
        self.optimizer.step()

In [ ]:
class LossWithLabelSmoothing(nn.Module):
    
    def __init__(self, size, smooth):
        super(LossWithLabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1 - smooth
        self.smooth = smooth
        self.size = size

    def forward(self, prediction, target, mask):
        """
        prediction of shape (batch_size, max_words, vocab_size)
        target and mask are of shape (batch_size, max_words)
        """
        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss


In [ ]:
d_model = 512
heads = 8
num_layers = 3
epochs = 10

with open('WORDMAP_corpus.json', 'r') as j:
    word_map = json.load(j)
    
transformer = Transformer(d_model = d_model, heads = heads, num_layers = num_layers, word_map = word_map)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLabelSmoothing(len(word_map), 0.1)

In [ ]:
def train(train_loader, transformer, criterion, epoch):
    
    transformer.train()
    sum_loss = 0
    count = 0

    for i, (question, reply) in enumerate(train_loader):
        
        samples = question.shape[0]

        # Move to device
        question = question.to(device)
        reply = reply.to(device)

        # Prepare Target Data
        reply_input = reply[:, :-1]
        reply_target = reply[:, 1:]

        # Create mask and add dimensions
        question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)

        # Get the transformer outputs
        out = transformer(question, question_mask, reply_input, reply_input_mask)

        # Compute the loss
        loss = criterion(out, reply_target, reply_target_mask)
        
        # Backprop
        transformer_optimizer.optimizer.zero_grad()
        loss.backward()
        transformer_optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        if i % 100 == 0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

In [ ]:
def evaluate(transformer, question, question_mask, max_len, word_map):
    """
    Performs Greedy Decoding with a batch size of 1
    """
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim = 1)
        next_word = next_word.item()
        if next_word == word_map['']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)   # (1,step+2)
        
    # Construct Sentence
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words if w not in {word_map['']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

In [ ]:
for epoch in range(epochs):
    
    train(train_loader, transformer, criterion, epoch)
    
    state = {'epoch': epoch, 'transformer': transformer, 'transformer_optimizer': transformer_optimizer}
    torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')

In [ ]:
checkpoint = torch.load('checkpoint.pth.tar')
transformer = checkpoint['transformer']

In [ ]:
while(1):
    question = input("Question: ") 
    if question == 'quit':
        break
    max_len = input("Maximum Reply Length: ")
    enc_qus = [word_map.get(word, word_map['']) for word in question.split()]
    question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
    question_mask = (question!=0).to(device).unsqueeze(1).unsqueeze(1)  
    sentence = evaluate(transformer, question, question_mask, int(max_len), word_map)
    print(sentence)